In [4]:
!pip install keras_vggface
!pip install keras_applications

In [2]:
from PIL import Image
#from keras_vggface.utils import preprocess_input
#from keras.utils.layer_utils import get_source_inputs
from keras_vggface.vggface import VGGFace
import numpy as np
import cv2
import os
from scipy import spatial
import pandas as pd 

In [3]:
BASE_MODEL = 'vgg16'
include_top = 'False'
pooling = 'avg'
model=VGGFace(model=BASE_MODEL,include_top=include_top,weights='vggface',input_shape=(224,224,3),pooling=pooling)
print(model)

In [10]:
def get_image_embedding(saving_dir):
    img=Image.open(saving_dir)
    img=img.resize((224,224))
    img_array=np.array(img).astype(np.float32)
    #img_array = img_array.transpose((2,0,1))
    expanded_img=np.expand_dims(img_array,axis=0)
    result=model.predict(expanded_img)
    return result.flatten()

In [11]:
def triplet_loss(e1,e2):
    #finds the difference between features of the each image
    distance=np.subtract(e1,e2)
    #the loss is squared so that non similar values stay more apart
    loss=np.square(distance) 
    return loss

In [12]:
import cv2
import sys

In [13]:
def Crop_Image(file_path,i,target_dir):
    image = cv2.imread(file_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(gray,scaleFactor=1.3,minNeighbors=3,minSize=(30, 30) )
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        roi_color = image[y:y + h, x:x + w]
        cv2.imwrite(target_dir+"//"+str(i)+'.jpg', roi_color)
#     cv2.imwrite(f"D:\Desktop\DLSIP\faces_on_original_image"+"//face_detected_{}")
#     status = cv2.imwrite(r"D:\Desktop\DLSIP\faces_on_original_image\faces_detected.jpg", image)
    #print("[INFO] Image faces_detected.jpg written to filesystem: ", status)

cleaning data

In [14]:
for i in range(1,16):
    path = os.path.join("D:\Desktop\DLSIP\dataset",f"person{i}")
    os.mkdir(f"D://Desktop//DLSIP//faces//person_{i}")
    images = os.listdir(path)
    for index,j in enumerate(images):
        image_path = os.path.join(path, j)
        Crop_Image(image_path,index,f"D://Desktop//DLSIP//faces//person_{i}")

embeddings


In [15]:
sets_of_similar_embbs=[]
for i in range(1,16):
    dir=r"D:\Desktop\DLSIP\faces\person_"+str(i)
    similarpics_embbs=[]
    for im in os.listdir(dir):
        j=os.path.join(dir,im)
        if os.path.isfile(j):
            e=get_image_embedding(j)
        similarpics_embbs.append(e)
    sets_of_similar_embbs.append(similarpics_embbs)

1/1 [==============================] - 0s 130ms/step


1/1 [==============================] - 0s 125ms/step


In [16]:
len(sets_of_similar_embbs[1])

7

creating dataset from embeddings


In [17]:
#no of features
features=['f'+str(i) for i in range(2622)]
df=pd.DataFrame(columns=features)

In [18]:
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f2612,f2613,f2614,f2615,f2616,f2617,f2618,f2619,f2620,f2621


In [19]:
for people in sets_of_similar_embbs:
    for i in range(0,len(people)-1):
        for j in range(i+1,len(people)):
            loss=triplet_loss(people[i],people[j])
            df=df.append(pd.DataFrame([loss], columns=features))

In [20]:
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f2612,f2613,f2614,f2615,f2616,f2617,f2618,f2619,f2620,f2621
0,1.468990e-05,9.145845e-06,7.561354e-11,6.233826e-10,6.036815e-08,4.271416e-05,3.340211e-10,1.182886e-08,3.121263e-10,3.915627e-09,...,1.541064e-09,3.955834e-15,3.183788e-16,8.766629e-11,7.854575e-13,3.253482e-15,1.617562e-15,1.589630e-18,1.876502e-09,2.251513e-08
0,1.890735e-05,7.952351e-07,4.661130e-12,6.116101e-11,6.278295e-12,4.609472e-05,2.869639e-08,2.397750e-11,4.974083e-10,7.046143e-12,...,7.968809e-10,2.592785e-19,3.455985e-19,1.068215e-09,2.903989e-13,2.607518e-19,3.559098e-15,5.684033e-21,4.797112e-13,1.396429e-13
0,4.537310e-06,6.380874e-07,2.465817e-11,4.248144e-11,3.632143e-10,4.525362e-05,1.171197e-07,4.673110e-11,3.298901e-10,8.977640e-13,...,7.894454e-10,4.819679e-17,6.840346e-18,4.356270e-10,4.971681e-13,1.063717e-16,7.338390e-16,1.397312e-18,1.600377e-11,1.779792e-11
0,6.268078e-06,5.719988e-06,5.391866e-11,7.210917e-11,1.356611e-11,6.161049e-06,4.223203e-08,1.051555e-08,3.359513e-10,4.336430e-13,...,2.620638e-09,7.673852e-19,8.642030e-20,7.840488e-11,8.831954e-14,3.146860e-19,7.507267e-15,2.670716e-18,1.861834e-13,5.473477e-10
0,1.799330e-06,6.700216e-06,3.795653e-12,8.013167e-11,2.223357e-12,5.655303e-05,6.632182e-05,4.478659e-11,5.307076e-10,1.113255e-11,...,2.640173e-09,2.548156e-18,1.456946e-21,1.461612e-09,3.696499e-14,7.410177e-18,7.372482e-15,1.504590e-19,3.991498e-14,2.058067e-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4.479586e-08,1.792738e-07,5.549207e-11,6.825563e-09,1.616016e-12,1.860821e-06,9.011777e-10,3.773557e-07,1.088401e-09,1.539255e-11,...,5.350088e-13,3.590536e-14,6.403132e-14,3.839881e-14,3.063048e-12,1.747543e-09,5.706249e-13,1.123171e-09,6.202325e-09,3.647329e-13
0,7.678789e-07,5.553151e-06,9.462388e-11,1.534336e-09,2.355581e-12,1.699726e-06,2.708071e-08,3.684806e-07,1.541544e-12,1.819115e-09,...,8.167770e-12,1.450194e-13,7.905831e-14,2.714397e-12,7.801916e-12,1.117630e-09,5.601751e-13,2.948450e-10,3.344782e-08,3.017766e-13
0,1.968741e-06,6.963236e-05,2.375139e-09,5.081547e-09,6.634622e-11,1.209389e-07,6.430555e-06,2.169901e-09,1.228584e-09,2.392913e-09,...,9.658902e-11,2.133287e-15,3.042471e-14,1.203322e-10,7.250180e-12,1.601238e-08,9.267239e-14,8.004805e-10,3.506996e-11,2.695711e-11
0,5.453550e-07,4.110729e-05,2.158273e-09,7.749887e-10,6.212206e-11,8.258662e-08,5.766296e-06,2.899710e-09,4.462306e-09,1.038291e-10,...,5.931284e-11,2.106548e-14,2.140304e-14,9.059264e-11,3.425208e-12,1.820143e-08,8.849001e-14,1.428043e-10,9.645325e-09,2.752694e-11


creating data frame false data

In [21]:
df1=pd.DataFrame(columns=features)

In [22]:
import random
for people in range(0,len(sets_of_similar_embbs)):
        random_rem=[n for n in range(0,len(sets_of_similar_embbs))]
        random_rem.remove(people)
        for iter in range(15):
            i=sets_of_similar_embbs[people]
            for q in range(20):  
                j = sets_of_similar_embbs[random.choice(random_rem)]
                loss=triplet_loss(random.choice(i),random.choice(j))
                df1=df1.append(pd.DataFrame([loss], columns=features))

In [23]:
trues=[1 for i in range(len(df))]
df['Target']=trues
falses=[0 for i in range(len(df1))]
df1['Target']=falses

In [24]:
df.to_csv(r"D:\Desktop\DLSIP\csv\file1.csv")

In [25]:
df1.to_csv(r"D:\Desktop\DLSIP\csv\file2.csv")

In [26]:
df1

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f2613,f2614,f2615,f2616,f2617,f2618,f2619,f2620,f2621,Target
0,2.468273e-07,2.206147e-06,6.241510e-08,3.308567e-11,2.632293e-05,2.307410e-06,1.990984e-09,5.378347e-10,5.745688e-11,7.993202e-12,...,1.338840e-13,5.562858e-15,8.708627e-10,1.486732e-10,6.571560e-15,8.517298e-13,8.821662e-16,1.064176e-12,6.679092e-08,0
0,2.583185e-05,8.900660e-03,8.531800e-11,2.806631e-13,4.037092e-13,2.863266e-05,6.694245e-10,4.906432e-10,1.153814e-11,1.497799e-13,...,1.237375e-14,5.817138e-17,3.153787e-09,1.245462e-12,2.280211e-13,2.225218e-13,6.327605e-15,2.640446e-11,1.028683e-11,0
0,1.748926e-05,2.917523e-04,5.446886e-09,5.640846e-10,2.696728e-10,4.287979e-05,2.871123e-05,8.145553e-12,6.691126e-09,6.019959e-09,...,4.313233e-15,2.972025e-15,1.028767e-09,2.699464e-15,9.381877e-14,1.865902e-15,4.287864e-16,2.768282e-10,2.901221e-10,0
0,6.761034e-05,5.337642e-03,2.816410e-08,5.236800e-10,3.349847e-11,4.047788e-06,3.876346e-09,1.529684e-10,4.367497e-08,1.151376e-08,...,6.819353e-16,6.322390e-16,5.495395e-10,6.868029e-14,3.854425e-13,5.470816e-17,1.535604e-14,6.598419e-10,3.219313e-10,0
0,1.045281e-06,5.983737e-06,8.095347e-08,6.169015e-10,2.494929e-10,2.825310e-05,1.932020e-07,4.027933e-05,1.059320e-10,4.896304e-10,...,3.649431e-15,3.210763e-11,3.915369e-08,7.902475e-12,2.160237e-09,2.674282e-12,2.340713e-15,4.891398e-08,1.332807e-09,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4.266342e-07,9.771225e-06,8.633257e-11,1.440094e-10,4.527106e-11,1.334412e-07,5.027341e-06,7.689560e-08,2.288967e-10,4.203891e-09,...,8.618105e-15,1.135943e-14,7.615330e-14,2.121264e-11,2.229450e-10,1.486636e-13,3.902542e-11,1.308251e-08,1.911566e-10,0
0,1.919883e-05,8.977227e-04,1.470830e-09,6.394037e-08,9.182742e-08,1.197772e-06,2.201042e-06,3.776596e-06,3.975823e-08,2.044439e-07,...,8.682058e-13,1.481869e-12,1.798863e-10,1.296874e-10,1.956742e-10,4.149199e-14,6.099022e-11,1.903825e-07,8.152316e-11,0
0,1.132087e-06,1.558934e-07,1.346467e-09,5.522298e-11,3.083914e-11,1.687944e-07,8.813958e-09,7.255239e-07,1.838682e-09,9.647055e-13,...,1.078775e-14,8.589287e-17,1.118413e-14,6.472440e-10,7.114235e-10,4.590944e-14,3.556808e-11,1.106363e-08,3.566987e-12,0
0,1.243751e-04,5.694845e-11,8.966305e-12,3.948985e-07,3.117309e-07,2.461410e-06,4.395226e-09,7.908272e-05,8.289216e-09,1.417672e-06,...,1.758047e-13,1.714297e-17,1.014223e-11,1.056532e-11,6.069141e-13,1.136239e-15,8.623689e-14,2.060230e-11,3.185108e-09,0


In [27]:
frames = [df, df1]

final = pd.concat(frames)

In [28]:
final.to_csv(r"D:\Desktop\DLSIP\csv\final.csv")

In [29]:
final

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f2613,f2614,f2615,f2616,f2617,f2618,f2619,f2620,f2621,Target
0,1.468990e-05,9.145845e-06,7.561354e-11,6.233826e-10,6.036815e-08,4.271416e-05,3.340211e-10,1.182886e-08,3.121263e-10,3.915627e-09,...,3.955834e-15,3.183788e-16,8.766629e-11,7.854575e-13,3.253482e-15,1.617562e-15,1.589630e-18,1.876502e-09,2.251513e-08,1
0,1.890735e-05,7.952351e-07,4.661130e-12,6.116101e-11,6.278295e-12,4.609472e-05,2.869639e-08,2.397750e-11,4.974083e-10,7.046143e-12,...,2.592785e-19,3.455985e-19,1.068215e-09,2.903989e-13,2.607518e-19,3.559098e-15,5.684033e-21,4.797112e-13,1.396429e-13,1
0,4.537310e-06,6.380874e-07,2.465817e-11,4.248144e-11,3.632143e-10,4.525362e-05,1.171197e-07,4.673110e-11,3.298901e-10,8.977640e-13,...,4.819679e-17,6.840346e-18,4.356270e-10,4.971681e-13,1.063717e-16,7.338390e-16,1.397312e-18,1.600377e-11,1.779792e-11,1
0,6.268078e-06,5.719988e-06,5.391866e-11,7.210917e-11,1.356611e-11,6.161049e-06,4.223203e-08,1.051555e-08,3.359513e-10,4.336430e-13,...,7.673852e-19,8.642030e-20,7.840488e-11,8.831954e-14,3.146860e-19,7.507267e-15,2.670716e-18,1.861834e-13,5.473477e-10,1
0,1.799330e-06,6.700216e-06,3.795653e-12,8.013167e-11,2.223357e-12,5.655303e-05,6.632182e-05,4.478659e-11,5.307076e-10,1.113255e-11,...,2.548156e-18,1.456946e-21,1.461612e-09,3.696499e-14,7.410177e-18,7.372482e-15,1.504590e-19,3.991498e-14,2.058067e-13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,4.266342e-07,9.771225e-06,8.633257e-11,1.440094e-10,4.527106e-11,1.334412e-07,5.027341e-06,7.689560e-08,2.288967e-10,4.203891e-09,...,8.618105e-15,1.135943e-14,7.615330e-14,2.121264e-11,2.229450e-10,1.486636e-13,3.902542e-11,1.308251e-08,1.911566e-10,0
0,1.919883e-05,8.977227e-04,1.470830e-09,6.394037e-08,9.182742e-08,1.197772e-06,2.201042e-06,3.776596e-06,3.975823e-08,2.044439e-07,...,8.682058e-13,1.481869e-12,1.798863e-10,1.296874e-10,1.956742e-10,4.149199e-14,6.099022e-11,1.903825e-07,8.152316e-11,0
0,1.132087e-06,1.558934e-07,1.346467e-09,5.522298e-11,3.083914e-11,1.687944e-07,8.813958e-09,7.255239e-07,1.838682e-09,9.647055e-13,...,1.078775e-14,8.589287e-17,1.118413e-14,6.472440e-10,7.114235e-10,4.590944e-14,3.556808e-11,1.106363e-08,3.566987e-12,0
0,1.243751e-04,5.694845e-11,8.966305e-12,3.948985e-07,3.117309e-07,2.461410e-06,4.395226e-09,7.908272e-05,8.289216e-09,1.417672e-06,...,1.758047e-13,1.714297e-17,1.014223e-11,1.056532e-11,6.069141e-13,1.136239e-15,8.623689e-14,2.060230e-11,3.185108e-09,0
